In [14]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Constants
WINDOW_SIZE = (600, 200)  # Window size for 1D environment
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 10
tolerance = 10  # How close the agent needs to be to a goal to trigger the end
noise_level = 15  # Noise level for second agent's movement

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("1D Environment with Moving Target")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Agent 1 (controlled by the user) starts in the middle of the line
agent1_pos = 300  # Middle of the line
reached_goal1 = False  # Track if agent 1 has reached a goal

# Agent 2 (autonomous) also starts in the middle
agent2_pos = 300
reached_goal2 = False  # Track if agent 2 has reached a goal

# Target (goal) is placed randomly on the 1D line
target_pos = random.randint(0, WINDOW_SIZE[0])  # Random position for the target on the line

# Y-position (vertical line) for both agents
line_y_position = 100  # Both agents will move on the same y-coordinate (horizontal line)

# Function to move agent 1 (controlled by the user)
def move_agent1(direction):
    global agent1_pos
    # Move based on direction
    if direction == "LEFT":
        new_pos = agent1_pos - step_size
    elif direction == "RIGHT":
        new_pos = agent1_pos + step_size

    # Ensure the agent stays within the window bounds
    if 0 <= new_pos <= WINDOW_SIZE[0]:
        agent1_pos = new_pos  # Update agent position if within bounds

    # Check if agent 1 has reached the target
    global reached_goal1
    reached_goal1 = abs(agent1_pos - target_pos) < tolerance

# Function to move agent 2 (autonomously following the target with noise)
def move_agent2():
    global agent2_pos
    # Calculate the difference between agent 2 and the target
    difference = target_pos - agent2_pos

    # Move in the direction of the target with added noise
    if difference > 0:
        new_pos = agent2_pos + step_size + random.uniform(-noise_level, noise_level)
    elif difference < 0:
        new_pos = agent2_pos - step_size + random.uniform(-noise_level, noise_level)

    # Ensure the agent stays within the window bounds
    if 0 <= new_pos <= WINDOW_SIZE[0]:
        agent2_pos = new_pos  # Update agent 2's position

    # Check if agent 2 has reached the target
    global reached_goal2
    reached_goal2 = abs(agent2_pos - target_pos) < tolerance

# Function to reset the environment
def reset():
    global agent1_pos, agent2_pos, reached_goal1, reached_goal2, target_pos
    agent1_pos = 300  # Reset agent 1 to the center
    agent2_pos = 300  # Reset agent 2 to the center
    reached_goal1 = False  # Reset goal flag for agent 1
    reached_goal2 = False  # Reset goal flag for agent 2
    target_pos = random.randint(0, WINDOW_SIZE[0])  # Place the target at a new random position

# Function to render the environment and GUI
def render():
    # Clear the screen
    screen.fill(WHITE)

    # Draw agent 1 (green circle) at the same y-coordinate
    pygame.draw.circle(screen, GREEN, (int(agent1_pos), line_y_position), 10)

    # Draw agent 2 (blue circle) at the same y-coordinate
    pygame.draw.circle(screen, BLUE, (int(agent2_pos), line_y_position), 10)

    # Draw the target (red circle) at the same y-coordinate
    pygame.draw.circle(screen, RED, (int(target_pos), line_y_position), 10)

    # If either agent reaches the target, display a message
    if reached_goal1 or reached_goal2:
        text = font.render("Target Reached! Press R to reset", True, FONT_COLOR)
        screen.blit(text, (150, 50))

    # Update the display
    pygame.display.update()

# Main loop
running = True
while running:
    moved = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:  # Reset the game if 'R' is pressed
                reset()

    # Check if agent 1 has reached a goal
    if not reached_goal1:  # Only allow movement if goal hasn't been reached
        # Get keys pressed
        keys = pygame.key.get_pressed()

        # Move based on keys
        if keys[pygame.K_LEFT]:      # Move strictly left
            move_agent1("LEFT")
            moved = True
        if keys[pygame.K_RIGHT]:     # Move strictly right
            move_agent1("RIGHT")
            moved = True

        # Only move agent 2 when agent 1 moves
        if moved and not reached_goal2:
            move_agent2()

    # Render the environment
    render()

    pygame.time.wait(100)  # Slow down for visibility

# Quit Pygame
pygame.quit()
